# Final exam programming problem

In this problem, you will work on the following:

- For a graph G, you will compute its MST
- We will add an edge into the MST and you should be able to quickly update the MST

To solve this problem, you may consult three lab sessions:

- Merge sort (We will sort the edges using merge sort)
- MST (Kruskal's algorithm)
- DFS (To find a cycle on a graph)

In [44]:
import random

## Part 1

You must work with the provided data structures.
You may use your own data structures if they are not defined here.

In [45]:
# Define graph edge
class Edge:
    def __init__(self, node1, node2, weight=1.0):
        self.node1 = node1
        self.node2 = node2
        self.weight = weight
        
    def __lt__(self, other):
        selfPriority = self.weight
        otherPriority = other.weight
        return selfPriority < otherPriority

In [46]:
# Define an undirected graph
class UndirectedGraph:
    def __init__(self, n):
        self.num_nodes = n
        self.nodes = [set() for i in range(n)]
        self.edges = []
    
    # edge node1 <--> node2 (undirected)
    def insert(self, node1, node2, weight=1.0):
        self.nodes[node1].add(node2)
        self.nodes[node2].add(node1)
        self.edges.append(Edge(node1, node2, weight))
        self.edges.append(Edge(node2, node1, weight))

### Graph representation

- Linked list: self.nodes in class UndirectedGraph (for DFS)
- Edge list: self.edges in class UndirectedGraph (for MST)

In [47]:
# A function to generate a random graph
def generate_graph_and_output_edges(num_nodes, random_seed, num_edge_per_node=3):
    random.seed(random_seed)
    graph = UndirectedGraph(num_nodes)
    edge_list = []
    for i in range(num_nodes):
        while len(graph.nodes[i]) < num_edge_per_node:
            node = random.randint(0, num_nodes-1)
            if node != i and (node not in graph.nodes[i]):
                weight = random.random()
                graph.insert(i, node, weight)
    return graph  

### A small sample graph

This is a sample graph so you can learn how graph is represented in this problem

In [48]:
# Generate the graph
graph_sample = generate_graph_and_output_edges(5, 100, 2)

# print nodes represented in linked list
print(graph_sample.nodes)

# print edges represented in edge list
for e in graph_sample.edges:
    print("{:2d}, {:2d}, {:.5f}".format(e.node1, e.node2, e.weight))

[{1, 2, 3}, {0, 3, 4}, {0, 3, 4}, {0, 1, 2}, {1, 2}]
 0,  1, 0.45953
 1,  0, 0.45953
 0,  3, 0.73196
 3,  0, 0.73196
 1,  3, 0.50687
 3,  1, 0.50687
 2,  0, 0.53290
 0,  2, 0.53290
 2,  3, 0.26342
 3,  2, 0.26342
 4,  1, 0.33535
 1,  4, 0.33535
 4,  2, 0.83890
 2,  4, 0.83890


In [49]:
# Union Find data structure is provided to support Kruskal
class UnionFind:
    def __init__(self, num_nodes):
        # Initially position[i] = i
        self.position = [i for i in range(num_nodes)]
        
    # Return the cluster index
    def find(self, node):
        if self.position[node] == node:
            return node
        else:
            self.position[node] = self.find(self.position[node])
            return self.position[node]
    
    def union(self, node1, node2):
        a = self.find(node1)
        b = self.find(node2)
        # no need to union
        if a == b:
            return
        # union is needed
        else:
            if a < b:
                self.position[b] = a
            else:
                self.position[a] = b

## Part 2

Find a MST for a given graph.

The returned MST should be in edge list format. (Just like what we did in the lab)

When sorting the graph.edges, you must use Merge Sort. (We used PriorityQueue in the lab)

In [50]:
# Implement MergeSort

# Your merge sort should returend a sorted list of A
def merge(X, Y):
    m = len(X)
    n = len(Y)
    sorted_array =[]
    #initilaize i and j
    i = 0
    j = 0
    
    #Compare edge wight from two array and append to sorted array
    while(i < m and j < n):
        if X[i].weight < Y[j].weight:
            sorted_array.append(X[i])
            i += 1
        else:
            sorted_array.append(Y[j])
            j += 1
    # add remaining edges to sorted array       
    while(i < m):
        sorted_array.append(X[i])
        i += 1
    while(j < n):
        sorted_array.append(Y[j])
        j += 1
        
    return sorted_array
    '''
    
    fill in the code
    
    '''

                
    
def MergeSort(A):
    n = len(A)
    if n <= 1:
        return A
    mid = n // 2
    #call merge sort recurrsively 
    l = MergeSort(A[:mid])
    r = MergeSort(A[mid:])
    
    return merge(l,r)
    '''
    
    fill in the code
    
    '''

In [51]:
# Implement Kruskal

# returned MST is a list of class Edge
def kruskal(graph):
    MST = []
    uf = UnionFind(graph.num_nodes)

    sorted_edge_list = MergeSort(graph.edges)
    
    s = len(graph.edges)
    
    while(len(MST) != s-1 and len(sorted_edge_list) > 0):
        edge = sorted_edge_list.pop(0)
        if (uf.find(edge.node1) != uf.find(edge.node2)):
            MST.append(edge)
            uf.union(uf.find(edge.node1), uf.find(edge.node2))
    return MST
    '''
    
    fill in the code
    
    '''

    
    return MST

## Part 3

A new edge (2, 19) with weight 0.2

We need to quickly update the old MST to get a new MST

In [52]:
# Construct a graph from edge list. This is given to you.

# We assume num_nodes is given for simplicity.
def generate_graph_from_edges(edge_list, num_nodes):
    graph = UndirectedGraph(num_nodes)
    for e in edge_list:
        graph.insert(e.node1, e.node2, e.weight)
    return graph

### Algorithm

- We add the new edge to the old MST which will generate a cycle

- The cycle consists at least three edges

- The edge with the largest weight should be removed

- You ONLY need to print out the edges on the cycle to get full marks (the two end nodes and the weight)

### Sample example

- We have a MST for a graph of 3 nodes

- MST has edge (0, 1, 0.12) and (0, 2, 0.23)

- We add a new edge (1, 2, 0.2)

- A cycle of three edges is generated

- We need to delete edge (0, 2, 0.23)

### How to use DFS to find the cycle

- Suppose the new edge is (2, 19, 0.2)

- If we DFS from node 2, we will eventurally hit node 19

- You need to properly append and pop on "cycle_edges" during DFS 

- The "cycle_edge" should contain the final path from 2 to 19

In [53]:
# Use DFS to detect the cycle

# You may add more arguments in dfs_recursive() for your convenience

def dfs(graph, starting_node, stopping_node):
    visited_nodes = [False for i in range(graph.num_nodes)]
    cycle_edges = []
    dfs_recursive(graph, starting_node, visited_nodes, cycle_edges, stopping_node)
    
    # The following is for demonstration purpose
#     print("We found {:d} edges from node {:d} to node {:d}"
#           .format(len(cycle_edges), starting_node, stopping_node))
#     for e in cycle_edges:
#         '''
#         print the two end nodes of the edge
#             the weight information is not known so you do not need to output 
        
#         You may change the following print statement
#             if you did not use Edge class
#         '''
#         print("{:2d}, {:2d}".format(e.node1, e.node2))


def dfs_recursive(graph, starting_node, visited_nodes, cycle_edges, stopping_node):
    if visited_nodes[starting_node]:
        return
    
    if starting_node == stopping_node:
        print("We found {:d} edges from node {:d} to node {:d}"
          .format(len(cycle_edges), starting_node, stopping_node))
        for e in cycle_edges:
            print("{:2d}, {:2d}".format(e.node1, e.node2))
        return
    visited_nodes[starting_node] = True
    
    node_list = list(graph.nodes[starting_node])
    
    for node in node_list:
        if not visited_nodes[node]:
            cycle_edges.append(Edge(starting_node,node))
            dfs_recursive(graph,node,visited_nodes,cycle_edges,stopping_node)
            cycle_edges.pop()
            
        
    
    '''
    
    fill in code here
    
    '''

## Part 4 Test code

Do NOT modify the code in this part.


In [54]:
graph_mst = generate_graph_and_output_edges(20, 10, 5)
MST = kruskal(graph_mst)

for e in MST:
    print("{:2d}, {:2d}, {:.5f}".format(e.node1, e.node2, e.weight))

18,  3, 0.00406
 1,  5, 0.03175
18,  0, 0.03259
 5,  0, 0.03440
11,  1, 0.04456
 2, 18, 0.05057
15, 13, 0.06277
15,  4, 0.06499
 7, 11, 0.07993
 4, 19, 0.10876
 8,  9, 0.14351
16,  5, 0.15642
 9, 10, 0.16494
14,  7, 0.16636
16,  9, 0.17846
 4,  3, 0.19495
 4,  6, 0.38442
 5, 12, 0.39059
 0, 17, 0.43858


In [55]:
def find_the_cycle(MST, new_edge, num_nodes):
    # construct graph from edge list
    graph_cycle = generate_graph_from_edges(MST, num_nodes)

    # call dfs to find a path from node1 to node2
    dfs(graph_cycle, new_edge.node1, new_edge.node2)

In [56]:
find_the_cycle(MST, Edge(2, 19, 0.2), 20)

We found 4 edges from node 19 to node 19
 2, 18
18,  3
 3,  4
 4, 19


In [57]:
# Instead we can add edge (3, 14, 0.1)
find_the_cycle(MST, Edge(3, 14, 0.1), 20)

We found 7 edges from node 14 to node 14
 3, 18
18,  0
 0,  5
 5,  1
 1, 11
11,  7
 7, 14


### Hint

If you run the following codes,

In [58]:
graph_hint = generate_graph_and_output_edges(10, 1, 2)

mst = kruskal(graph_hint)

print("The minimum spanning tree:")
for e in mst:
    print("{:2d}, {:2d}, {:.5f}".format(e.node1, e.node2, e.weight))

find_the_cycle(mst, Edge(1, 8, 0.2), 10)

The minimum spanning tree:
 8,  6, 0.00920
 0,  5, 0.02232
 1,  0, 0.25507
 6,  3, 0.43277
 1,  3, 0.48786
 0,  9, 0.52763
 2,  0, 0.56920
 3,  4, 0.59115
 7,  2, 0.65159
We found 3 edges from node 8 to node 8
 1,  3
 3,  6
 6,  8


In [ ]:
# this is the results you should get:
'''
The minimum spanning tree:
 8,  6, 0.00920
 0,  5, 0.02232
 1,  0, 0.25507
 6,  3, 0.43277
 1,  3, 0.48786
 0,  9, 0.52763
 2,  0, 0.56920
 3,  4, 0.59115
 7,  2, 0.65159
We found 3 edges from node 1 to node 8
 1,  3
 3,  6
 6,  8
'''
print()